<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/CSV_to_UVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hier wird mit Hilfe der Messdaten der UVI berechnet

In [121]:
# Import der benötigten Module
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
from google.colab import files

In [122]:
# Mount google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [123]:
# Monat der importiert werden soll
month = '22.07'

In [124]:
# Import der CSV mit den rohen Messdaten
# Pfad zur CSV-Datei auf Google Drive
drive_path = '/content/drive/My Drive/Colab_Notebooks/CSV_Messdaten/' + month

# CSV-Datei einlesen und DataFrame erstellen
df_Messdaten = pd.read_csv(drive_path)

In [125]:
df_Messdaten

,Datetime,Datum,Uhrzeit,Messzeitpunkt,290.0,290.1,290.2,290.3,290.4,290.5,...,419.1,419.2,419.3,419.4,419.5,419.6,419.7,419.8,419.9,420.0
0,2022-07-01 03:27:00,2022-07-01,03:27:00,12420,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005206,0.005186,0.005160,0.005053,0.004953,0.004876,0.004833,0.004813,0.004802,0.004844
1,2022-07-01 03:29:00,2022-07-01,03:29:00,12540,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005520,0.005478,0.005376,0.005309,0.005241,0.005140,0.005096,0.005084,0.005104,0.005163
2,2022-07-01 03:31:00,2022-07-01,03:31:00,12660,0.0,0.0,0.0,0.0,0.0,0.0,...,0.006334,0.006310,0.006229,0.006150,0.006064,0.005986,0.005920,0.005857,0.005820,0.005897
3,2022-07-01 03:33:00,2022-07-01,03:33:00,12780,0.0,0.0,0.0,0.0,0.0,0.0,...,0.007632,0.007576,0.007482,0.007331,0.007234,0.007116,0.007061,0.007023,0.007019,0.007095
4,2022-07-01 03:35:00,2022-07-01,03:35:00,12900,0.0,0.0,0.0,0.0,0.0,0.0,...,0.009206,0.009142,0.009061,0.008908,0.008740,0.008574,0.008480,0.008461,0.008488,0.008570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13539,2022-07-31 18:47:00,2022-07-31,18:47:00,67620,0.0,0.0,0.0,0.0,0.0,0.0,...,0.028136,0.028055,0.027764,0.027342,0.026629,0.026046,0.025700,0.025493,0.025525,0.025941
13540,2022-07-31 18:49:00,2022-07-31,18:49:00,67740,0.0,0.0,0.0,0.0,0.0,0.0,...,0.025959,0.025925,0.025614,0.025066,0.024481,0.024037,0.023801,0.023748,0.023842,0.023911
13541,2022-07-31 18:51:00,2022-07-31,18:51:00,67860,0.0,0.0,0.0,0.0,0.0,0.0,...,0.023252,0.023143,0.022897,0.022585,0.022080,0.021822,0.021536,0.021379,0.021455,0.021605
13542,2022-07-31 18:53:00,2022-07-31,18:53:00,67980,0.0,0.0,0.0,0.0,0.0,0.0,...,0.020673,0.020705,0.020372,0.019932,0.019637,0.019423,0.019186,0.019057,0.018899,0.019209


In [126]:
# Benutzerdefinierte Gewichtungsfunktion
def weight_function(Wellenlaenge):
    if Wellenlaenge <= 298:
        return 1
    elif Wellenlaenge > 298 and Wellenlaenge <= 328:
        return 10**(0.094 * (298.0 - Wellenlaenge))
    elif Wellenlaenge > 328 and Wellenlaenge <= 400:
        return 10**(0.015 * (139.0 - Wellenlaenge))
    else:
        return 0

In [127]:
# Anwenden der Gewichtungsfunktion auf den DataFrame
# Bereich von Spalte x bis Spalte y (einschließlich y)
x = '290.0'
y = '400.0'
selected_columns = df_Messdaten.columns[(df_Messdaten.columns.get_loc(x)):(df_Messdaten.columns.get_loc(y)+1)]

In [128]:
df_Gewichtet = []
df_Gewichtet = df_Messdaten[selected_columns].apply(lambda col: col * weight_function(float(col.name)), axis=0)

In [129]:
def integrate_row(row):
    x_values = np.array([float(col) for col in row.index])  # X-Werte aus Spaltennamen extrahieren
    y_values = np.array(row)
    integral = np.trapz(y_values, x_values)
    return integral

In [130]:
ser_UVI = []
df_UVI = []

ser_UVI = df_Gewichtet.apply(integrate_row, axis=1)
df_UVI = pd.DataFrame(ser_UVI, columns=['UVI'])

In [131]:
type(df_Gewichtet), type(df_UVI)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [132]:
# Exportieren
columns_to_copy = ['Messzeitpunkt', 'Uhrzeit', 'Datum', 'Datetime']

for col in columns_to_copy:
  df_Gewichtet.insert(0, col, df_Messdaten[col])
  df_UVI.insert(0, col, df_Messdaten[col])

In [133]:
df_UVI

,Datetime,Datum,Uhrzeit,Messzeitpunkt,UVI
0,2022-07-01 03:27:00,2022-07-01,03:27:00,12420,0.000156
1,2022-07-01 03:29:00,2022-07-01,03:29:00,12540,0.000168
2,2022-07-01 03:31:00,2022-07-01,03:31:00,12660,0.000192
3,2022-07-01 03:33:00,2022-07-01,03:33:00,12780,0.000227
4,2022-07-01 03:35:00,2022-07-01,03:35:00,12900,0.000282
...,...,...,...,...,...
13539,2022-07-31 18:47:00,2022-07-31,18:47:00,67620,0.000891
13540,2022-07-31 18:49:00,2022-07-31,18:49:00,67740,0.000807
13541,2022-07-31 18:51:00,2022-07-31,18:51:00,67860,0.000727
13542,2022-07-31 18:53:00,2022-07-31,18:53:00,67980,0.000645


In [134]:
def save_dataframe_to_drive(dataframe, folder_path, filename):

    # Erstelle den vollen Pfad zur Datei
    full_path = os.path.join(folder_path, filename)

    # Speichere den DataFrame als CSV-Datei auf Google Drive
    dataframe.to_csv(full_path, index=False)

    print(f'Der DataFrame wurde als {filename} in {folder_path} auf Google Drive gespeichert.')

In [ ]:
plt.plot(df_UVI)

In [135]:
save_dataframe_to_drive(df_UVI, '/content/drive/My Drive/Colab_Notebooks/CSV_UVI', month)

Der DataFrame wurde als 22.07 in /content/drive/My Drive/Colab_Notebooks/CSV_UVI auf Google Drive gespeichert.


In [ ]:
# save_dataframe_to_drive(df_Gewichtet, '/content/drive/My Drive/Colab_Notebooks/CSV_Gewichtet', month)